In [1]:
import sys
sys.path.insert(0, '../..')
from src.data_process.prep_batch_for_annot import *
from src.utils.latex import show_latex, TABLES

In [2]:
import pandas as pd
from functools import partial
from pathlib import Path

# Load data

In [3]:
cov_amc_path = '../../../Covid_data_11nov/raw/ICD_U07.1/a-proof-zonmw/amc_[U07.1]_2020_q1_q2_q3.csv'
cov_vumc_path = '../../../Covid_data_11nov/raw/ICD_U07.1/a-proof-zonmw/vumc_[U07.1]_2020_q1_q2_q3.csv'
annotated = '../../../Covid_data_11nov/from_inception_tsv/annotated_notes_ids.csv'

In [4]:
cov_amc = pd.read_csv(cov_amc_path, index_col=0)
cov_vumc = pd.read_csv(cov_vumc_path, index_col=0)

In [5]:
cov_amc['all_text'] = cov_amc.iloc[:,-3:].apply(lambda s: ' '.join([i for i in s.values if i==i]), axis=1)
cov_vumc['all_text'] = cov_vumc.iloc[:,-3:].apply(lambda s: ' '.join([i for i in s.values if i==i]), axis=1)

In [6]:
annotated = pd.read_csv(annotated)

# Load keywords

In [7]:
kwd = pd.read_excel('../../keywords/keywords.xlsx')
kwd['regex'] = kwd.apply(lambda row: get_regex(row.keyword, row.regex_template_id), axis=1)
kwd

,domain,keyword,regex_template_id,regex
0,ENR,energie,0,\benergie.*?\b
1,ENR,vermoei,0,\bvermoei.*?\b
2,ENR,oververmoei,0,\boververmoei.*?\b
3,ENR,moe,1,\bmoe\b
4,ENR,uitgeput,0,\buitgeput.*?\b
...,...,...,...,...
120,BER,ontsla,0,\bontsla.*?\b
121,BER,baas,1,\bbaas\b
122,BER,manager,1,\bmanager\b
123,BER,stud,0,\bstud.*?\b


In [8]:
reg_dict = get_reg_dict(kwd)

# Find keywords

In [9]:
cov_amc = find_keywords(cov_amc, reg_dict)

In [10]:
cov_vumc = find_keywords(cov_vumc, reg_dict)

# Stats

In [11]:
def combine_dfs(list_of_dfs, list_of_hosp_names):
    return pd.concat(list_of_dfs, keys=list_of_hosp_names, names=['institution', 'idx_source_file'])

In [12]:
domains = ['ENR', 'ATT', 'STM', 'ADM', 'INS', 'MBW', 'FAC', 'BER']
matched_domains = [f"matched_{domain}" for domain in domains]
count_domains = [f"n_{domain}" for domain in domains]

def op_count(df, domain):
    "Number of matches for `domain` keywords."
    return df[domain].apply(len)

def op_bool(df, domain):
    "Are there any matches for `domain` keywords (boolean)."
    return df[domain].apply(bool)

ops_count = {f"n_{domain}":partial(op_count, domain=domain) for domain in domains}
ops_bool = {f"matched_{domain}":partial(op_bool, domain=domain) for domain in domains}

In [24]:
results = combine_dfs([cov_amc, cov_vumc], ['AMC', 'VUmc']
).reset_index(
).merge(annotated.iloc[:,:-1], on=['institution', 'MDN', 'NotitieID', 'NotitieCSN'], how='left'
).set_index('idx_source_file'
).drop_duplicates(subset=['MDN', 'NotitieID', 'all_text'], keep='first'
).assign(**ops_count, **ops_bool
).assign(
    kwd_match=lambda df: df[domains].any(axis=1),
    batch=lambda df: df.batch.fillna('not annotated'),
    n_domains=lambda df: df[matched_domains].sum(axis=1)
)

In [26]:
caption = "Num. notes with/without keyword matches in the COVID data" 
label = "kwd_covid_overview"

results.pivot_table(
    index=['batch','institution',],
    columns=['kwd_match'],
    values='NotitieID',
    aggfunc='count',
    margins=True,
    margins_name='Totals',
).pipe(show_latex, caption, label)

kwd_match                     False    True  Totals
batch            institution                       
not annotated    AMC          4,486  15,976  20,462
                 VUmc         4,591  16,037  20,628
pilot_CovidBatch AMC             34     907     941
                 VUmc            34     714     748
Totals                        9,145  33,634  42,779

In [27]:
q_no_ann = "batch == 'not annotated'"
q_match = "n_domains > 0"

In [28]:
caption = "Domains overview: num. notes with at least one keyword match" 
label = "kwd_covid_matched_domains"

results.query(q_no_ann).pivot_table(
    index=['institution',],
    values=matched_domains,
    aggfunc='sum',
    margins=True,
    margins_name='Totals',
).T.sort_values('Totals', ascending=False).pipe(show_latex, caption, label)

institution,AMC,VUmc,Totals
matched_ADM,"11,542","10,768","22,310"
matched_MBW,"8,899","8,824","17,723"
matched_BER,"5,271","4,660","9,931"
matched_FAC,"4,242","4,272","8,514"
matched_ENR,"3,734","3,735","7,469"
matched_INS,"2,705","3,424","6,129"
matched_STM,"2,680","2,925","5,605"
matched_ATT,"2,444","2,362","4,806"


In [29]:
caption = " Mean/median/max num. keyword matches per domain" 
label = "kwd_covid_kwd_per_dom"

dfs = [results.query(f"n_{dom} > 0")[f"n_{dom}"].agg(['mean', 'median', 'max']) for dom in domains]
pd.concat(dfs, keys=domains).unstack(1).sort_values('mean', ascending=False).round(2
).astype({
    'median': int,
    'max': int,
}).pipe(show_latex, caption, label)

,mean,median,max
ADM,3.32,2,39
MBW,3.15,2,134
BER,2.03,1,22
INS,1.99,1,23
FAC,1.87,1,27
ENR,1.86,1,13
STM,1.58,1,31
ATT,1.4,1,13


In [30]:
caption = "Number of matched domains per note" 
label = "kwd_covid_n_matched domains"

results.query(q_no_ann).query(q_match).pivot_table(
    columns=['institution',],
    index=['n_domains'],
    aggfunc='count',
    values='NotitieID',
    margins=True,
    margins_name='Totals',
).pipe(show_latex, caption, label)

institution,AMC,VUmc,Totals
n_domains,,,
1,"4,722","4,812","9,534"
2,"4,183","4,272","8,455"
3,"3,146","3,238","6,384"
4,"2,011","1,913","3,924"
5,"1,065","1,013","2,078"
6,473,453,926
7,224,223,447
8,152,113,265
Totals,"15,976","16,037","32,013"


In [31]:
prefix = 'kwd_covid'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)